<a href="https://colab.research.google.com/github/ffffff-990917/2022-1-Euron-Study-Assignments/blob/Week_6/week5_nlp_hw_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

📌 week5 내용 주차에 해당되는 과제는 3주차의 Glove 모델 실습, 4주차의 NER task 실습, 5주차의 Dependency Parsing task 실습으로 구성되어 있습니다. (**참고** : 제출은 week6 branch 복습과제로!) 

📌 위키독스의 딥러닝을 이용한 자연어 처리 입문 교재 실습, 캐글 노트북 등의 자료로 구성되어있는 과제입니다. 

📌 안내된 링크에 맞추어 **직접 코드를 따라 치면서 (필사)** 해당 nlp task 의 기본적인 라이브러리와 메서드를 숙지해보시면 좋을 것 같습니다😊 필수라고 체크한 부분은 과제에 반드시 포함시켜주시고, 선택으로 체크한 부분은 자율적으로 스터디 하시면 됩니다.

📌 궁금한 사항은 깃허브 이슈나, 카톡방, 세션 발표 시작 이전 시간 등을 활용하여 자유롭게 공유해주세요!

In [7]:
import nltk
# nltk colab 환경에서 실행시 필요한 코드입니다. 
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

### 1️⃣ **Glove**



👀 **내용 복습** 
* 스탠포드 대학에서 개발한 카운트 기반과 예측 기반을 모두 사용하는 단어 임베딩 방법론 
* word2vec 의 단점을 보완해서 나온 모델 
* glove model 의 **input 은 반드시 동시등장행렬 형태**여야 한다 ⭐

![1](https://www.dropbox.com/s/nz0ji4yzre56ifv/word_presentation.png?raw=1) 




🤔 한국어 예제는 없는 것 같습니다. 논문에서는 k-Glove 로 소개되는 연구가 있긴 한데, 좀 더 알아봐야 할 것 같아요!

➕ [논문1](https://scienceon.kisti.re.kr/srch/selectPORSrchArticle.do?cn=NPAP13255003&dbt=NPAP)


➕[논문2](https://scienceon.kisti.re.kr/commons/util/originalView.do?cn=CFKO201832073078664&oCn=NPAP13255064&dbt=CFKO&journal=NPRO00383361&keyword=%ED%95%9C%EA%B5%AD%EC%96%B4%20%EB%8C%80%ED%99%94%20%EC%97%94%EC%A7%84%EC%97%90%EC%84%9C%EC%9D%98%20%EB%AC%B8%EC%9E%A5%EB%B6%84%EB%A5%98)

🔹 **1-(1)** glove python

* [실습 : basic code](https://wikidocs.net/22885) 👉 필수

In [3]:
!pip install glove_python_binary


     |████████████████████████████████| 948 kB 7.6 MB/s 


In [4]:
import re
import urllib.request
import zipfile
from lxml import etree
from nltk.tokenize import word_tokenize, sent_tokenize

In [5]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/09.%20Word%20Embedding/dataset/ted_en-20160408.xml", filename="ted_en-20160408.xml")

('ted_en-20160408.xml', <http.client.HTTPMessage at 0x7f9e3962f050>)

In [6]:
targetXML = open('ted_en-20160408.xml', 'r', encoding='UTF8')
target_text = etree.parse(targetXML)

# xml 파일로부터 <content>와 </content> 사이의 내용만 가져온다.
parse_text = '\n'.join(target_text.xpath('//content/text()'))

# 정규 표현식의 sub 모듈을 통해 content 중간에 등장하는 (Audio), (Laughter) 등의 배경음 부분을 제거.
# 해당 코드는 괄호로 구성된 내용을 제거.
content_text = re.sub(r'\([^)]*\)', '', parse_text)

# 입력 코퍼스에 대해서 NLTK를 이용하여 문장 토큰화를 수행.
sent_text = sent_tokenize(content_text)

# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.
normalized_text = []
for string in sent_text:
     tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
     normalized_text.append(tokens)

# 각 문장에 대해서 NLTK를 이용하여 단어 토큰화를 수행.
result = [word_tokenize(sentence) for sentence in normalized_text]

KeyboardInterrupt: ignored

In [ ]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

model = Word2Vec(sentences=result, size=100, window=5, min_count=5, workers=4, sg=0)


In [ ]:
from glove import Corpus, Glove

corpus = Corpus() 

# 훈련 데이터로부터 GloVe에서 사용할 동시 등장 행렬 생성
corpus.fit(result, window=5)
glove = Glove(no_components=100, learning_rate=0.05)

# 학습에 이용할 쓰레드의 개수는 4로 설정, 에포크는 20.
glove.fit(corpus.matrix, epochs=20, no_threads=4, verbose=True)
glove.add_dictionary(corpus.dictionary)

In [ ]:
print(glove.most_similar("man"))

In [ ]:
print(glove.most_similar("boy"))

In [ ]:
print(glove.most_similar("university"))

In [ ]:
print(glove.most_similar("water"))

In [ ]:
print(glove.most_similar("physics"))

In [ ]:
print(glove.most_similar("muscle"))

In [ ]:
print(glove.most_similar("clean"))

🔹 **1-(2)** pre-trained glove 

* **사전학습모델** : 임의의 값으로 초기화하던 모델의 가중치들을 다른 문제에 학습시킨 가중치들로 초기화하는 방법이다.사전 학습한 가중치를 활용해 학습하고자 하는 본래 문제를 하위문제라고 한다. 

* [실습 : 문장의 긍부정을 판단하는 감성 분류 모델 만들기](https://wikidocs.net/33793) 👉 필수
  * [설명참고](https://omicro03.medium.com/%EC%9E%90%EC%97%B0%EC%96%B4%EC%B2%98%EB%A6%AC-nlp-16%EC%9D%BC%EC%B0%A8-pre-trained-word-embedding-bb30db424a35)
* pre-trained data 를 가져오는데 시간이 오래걸림
* kaggle 대회에서 주로 이 방식을 많이 사용함
  * [참고](https://lsjsj92.tistory.com/455)

In [ ]:
from keras.layers import Embedding

In [ ]:
vocab_size = 20000
output_dim = 128
input_length = 500

v = Embedding(vocab_size, output_dim, input_length = input_length)

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

sentences = ['nice great best amazing', 'stop lies', 'pitiful nerd', 'excellent work', 'supreme quality', 'bad', 'highly respectable']
y_train = [1, 0, 0, 1, 1, 0, 1]

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
vocab_size = len(tokenizer.word_index) + 1 # 패딩을 고려하여 +1
print('단어 집합 :',vocab_size)


In [ ]:
X_encoded = tokenizer.texts_to_sequences(sentences)
print('정수 인코딩 결과 :',X_encoded)

In [ ]:
max_len = max(len(l) for l in X_encoded)
print('최대 길이 :',max_len)

In [ ]:
X_train = pad_sequences(X_encoded, maxlen=max_len, padding='post')
y_train = np.array(y_train)
print('패딩 결과 :')
print(X_train)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten

embedding_dim = 4

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.fit(X_train, y_train, epochs=100, verbose=2)

In [ ]:
print(X_train)

In [ ]:
print(y_train)

In [ ]:
from urllib.request import urlretrieve, urlopen
import gzip
import zipfile

urlretrieve("http://nlp.stanford.edu/data/glove.6B.zip", filename="glove.6B.zip")
zf = zipfile.ZipFile('glove.6B.zip')
zf.extractall() 
zf.close()

In [ ]:
embedding_dict = dict()

f = open('glove.6B.100d.txt', encoding="utf8")

for line in f:
    word_vector = line.split()
    word = word_vector[0]

    # 100개의 값을 가지는 array로 변환
    word_vector_arr = np.asarray(word_vector[1:], dtype='float32')
    embedding_dict[word] = word_vector_arr
f.close()

print('%s개의 Embedding vector가 있습니다.' % len(embedding_dict))

In [ ]:
print(embedding_dict['respectable'])
print('벡터의 차원 수 :',len(embedding_dict['respectable']))

In [ ]:
embedding_matrix = np.zeros((vocab_size, 100))
print('임베딩 행렬의 크기(shape) :',np.shape(embedding_matrix))

In [ ]:
print(tokenizer.word_index.items())

In [ ]:
print('단어 great의 맵핑된 정수 :',tokenizer.word_index['great'])

In [ ]:
print(embedding_dict['great'])

In [ ]:
for word, index in tokenizer.word_index.items():
    # 단어와 맵핑되는 사전 훈련된 임베딩 벡터값
    vector_value = embedding_dict.get(word)
    if vector_value is not None:
        embedding_matrix[index] = vector_value

In [ ]:
embedding_matrix[2]

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten

output_dim = 100

model = Sequential()
e = Embedding(vocab_size, output_dim, weights=[embedding_matrix], input_length=max_len, trainable=False)
model.add(e)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.fit(X_train, y_train, epochs=100, verbose=2)

🔹 **1-(3)** fine tuning glove
* 미세조정 : 사전 학습한 모든 가중치와 더불어 하위 문제를 위한 최소한의 가중치를 추가해 모델을 추가로 학습하는 방법이다. 

* fine tuning 이 필요한 경우 
  * pretrained model 에 데이터셋에 있는 단어가 포함되지 않은 경우 
  * 데이터 집합이 너무 작아서 전체 모델을 훈련시키기 어려운 경우 

* [Mittens 라이브러리로 fine tuning](https://towardsdatascience.com/fine-tune-glove-embeddings-using-mittens-89b5f3fe4c39) 👉 필수
  *  GloVe 임베딩을 fine-tuning 하기 위한 파이썬 라이브러리
  * [github](https://github.com/roamanalytics/mittens)

* [한국어 소설 텍스트 데이터 미세조정 모델 학습 - GPT2](https://m.blog.naver.com/PostView.nhn?isHttpsRedirect=true&blogId=horajjan&logNo=222104684132&categoryNo=120&proxyReferer=) 👉 선택 (glove 모델 예제는 아닙니다. fine-tuning 에 초점을 두어서 참고해주시면 좋을 것 같습니다.)

In [ ]:
pip install -U mittens

In [ ]:
import csv
import numpy as np
from collections import Counter
from nltk.corpus import brown
nltk.download('brown')
from mittens import GloVe, Mittens
from sklearn.feature_extraction import _stop_words
from sklearn.feature_extraction.text import CountVectorizer
import pickle


def glove2dict(glove_filename):
    with open(glove_filename, encoding='utf-8') as f:
        reader = csv.reader(f, delimiter=' ', quoting=csv.QUOTE_NONE)
        embed = {line[0]: np.array(list(map(float, line[1:])))
                for line in reader}
    return embed

glove_path = "glove.6B.50d.txt" # get it from https://nlp.stanford.edu/projects/glove
pre_glove = glove2dict(glove_path)

sw = list(_stop_words.ENGLISH_STOP_WORDS)
brown_data = brown.words()[:200000]
brown_nonstop = [token.lower() for token in brown_data if (token.lower() not in sw)]
oov = [token for token in brown_nonstop if token not in pre_glove.keys()]

def get_rareoov(xdict, val):
    return [k for (k,v) in Counter(xdict).items() if v<=val]

#oov_rare = get_rareoov(oov, 1)
#corp_vocab = list(set(oov) - set(oov_rare))
#brown_tokens = [token for token in brown_nonstop if token not in oov_rare]
#brown_doc = [' '.join(brown_tokens)]

corp_vocab = list(set(oov))
brown_doc = [' '.join(brown_nonstop)]

cv = CountVectorizer(ngram_range=(1,1), vocabulary=corp_vocab)
X = cv.fit_transform(brown_doc)
Xc = (X.T * X)
Xc.setdiag(0)
coocc_ar = Xc.toarray()

mittens_model = Mittens(n=50, max_iter=1000)

new_embeddings = mittens_model.fit(
    coocc_ar,
    vocab=corp_vocab,
    initial_embedding_dict= pre_glove)

newglove = dict(zip(corp_vocab, new_embeddings))
f = open("repo_glove.pkl","wb")
pickle.dump(newglove, f)
f.close()

* (참고) word2vec pretrained example

➕ [word2vec 사전학습 모델 -한국어1](http://doc.mindscale.kr/km/unstructured/11.html)

➕ [word2vec 사전학습 - 한국어2](https://monetd.github.io/python/nlp/Word-Embedding-Word2Vec-%EC%8B%A4%EC%8A%B5/#%ED%95%9C%EA%B5%AD%EC%96%B4-word2vec-%EB%A7%8C%EB%93%A4%EA%B8%B0)

### **2️⃣ NER**

👀 **내용 복습** 
* 개체명 인식을 사용하면 코퍼스로부터 어떤 단어가 사람, 장소, 조직 등을 의미하는 단어인지를 찾을 수 있다. 




🔹 **2-(1)** NER task by nltk library


* nltk 에서는 개체명 인식기 (NER chunker) 를 지원하고 있다. 
* ne_chunk 는 개체명을 태깅하기 위해서 앞서 품사 태깅 pos_tag 가 수행되어야 한다. 


📌 [basic code](https://wikidocs.net/30682) 👉 필수 

📌 [BIO 표현, LSTM을 활용한 NER 실습](https://wikidocs.net/24682) 👉 선택




In [8]:
from nltk import word_tokenize, pos_tag, ne_chunk

sentence = "James is working at Disney in London"
# 토큰화 후 품사 태깅
tokenized_sentence = pos_tag(word_tokenize(sentence))
print(tokenized_sentence)

[('James', 'NNP'), ('is', 'VBZ'), ('working', 'VBG'), ('at', 'IN'), ('Disney', 'NNP'), ('in', 'IN'), ('London', 'NNP')]


In [9]:
# 개체명 인식
ner_sentence = ne_chunk(tokenized_sentence)
print(ner_sentence)

(S
  (PERSON James/NNP)
  is/VBZ
  working/VBG
  at/IN
  (ORGANIZATION Disney/NNP)
  in/IN
  (GPE London/NNP))


🔹 **2-(2)** NER task by spacy library


* spaCy 는 자연어처리를 위한 파이썬 기반의 오픈 소스 라이브러리로 다음과 같은 기능을 제공한다. 
  * Tokenization 
  * POS tagging 
  * Lemmatization 
  * Sentence Boundary Detection (SBD)
  * Named Entity Recognition (NER)
  * Similarity
  * Text Classification
  * Rule-based Matching
  * Training
  * Serialization

* spaCy 와 NER
  * .ents → .label_


📌 [basic code](https://frhyme.github.io/python-lib/nlp_spacy_1/) 👉 필수 (NER 부분만)

📌 [kaggle_Custom NER using SpaCy](https://www.kaggle.com/code/amarsharma768/custom-ner-using-spacy/notebook) 👉 선택

  * 훈련되지 않은 데이터 세트에 명명된 엔티티를 학습하는 방법 : 이력서 pdf 데이터 활용 
  * manually labelled 

📌 [한국어 NER](https://github.com/monologg/KoBERT-NER) 👉 참고하면 좋을 자료

➕ [참고](http://aispiration.com/nlp2/nlp-ner-python.html)

In [20]:
!pip install spacytextblob
!python -m textblob.download_corpora
!python -m spacy download en_core_web_sm

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.
Finished.
     |████████████████████████████████| 13.9 MB 8.5 MB/s 
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 2.2.5
    Uninstalling en-core-web-sm-2.2.5:
      Successfully uninstalled en-core-web-sm-2.2.5
✔ Downloa

In [21]:
import spacy
nlp = spacy.load('en_core_web_sm')
doc = nlp('Apple is looking at buying U.K. startup for $1 billion')

for ent in doc.ents:
    print(ent.text, ent.label_)

Apple ORG
U.K. GPE
$1 billion MONEY


In [22]:
doc = nlp("""But Google is starting from behind. The company made a late push
into hardware, and Apple’s Siri, available on iPhones, and Amazon’s Alexa
software, which runs on its Echo and Dot devices, have clear leads in
consumer adoption.""".replace("\n", " ").strip())

for ent in doc.ents:
    print(ent.text, ent.label_)

Google ORG
Apple’s Siri ORG
iPhones ORG
Amazon’s Alexa ORG
Echo GPE


In [23]:
doc = nlp("This is a sentence. This is another sentence.")
for sent in doc.sents:
    print(sent.text)

This is a sentence.
This is another sentence.


In [24]:
!python -m spacy download en_core_web_md
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 45.7 MB 2.1 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
     |████████████████████████████████| 777.4 MB 6.7 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [25]:
import spacy

nlp = spacy.load('en_core_web_md')  # make sure to use larger model!
tokens = nlp(u'dog cat banana')

for token1 in tokens:
    for token2 in tokens:
        print(token1.text, token2.text, token1.similarity(token2))

dog dog 1.0
dog cat 0.8016854524612427
dog banana 0.2432764321565628
cat dog 0.8016854524612427
cat cat 1.0
cat banana 0.28154364228248596
banana dog 0.2432764321565628
banana cat 0.28154364228248596
banana banana 1.0


In [26]:
import spacy
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances

nlp = spacy.load('en_core_web_md')  # make sure to use larger model!
tokens = nlp(u'dog cat banana')

for token1 in tokens:
    for token2 in tokens:
        ## np.dot(token1.vector, token2.vector) / (token1.vector_norm*token2.vector_norm)
        sim = cosine_similarity(token1.vector.reshape(1, 300), 
                          token2.vector.reshape(1, 300)
                         )
        print(token1.text, token2.text, sim)

dog dog [[0.99999994]]
dog cat [[0.8016855]]
dog banana [[0.24327646]]
cat dog [[0.8016855]]
cat cat [[1.]]
cat banana [[0.28154367]]
banana dog [[0.24327646]]
banana cat [[0.28154367]]
banana banana [[1.]]


###**3️⃣ Dependency Parsing**

👀 **내용 복습** 
* 문장의 전체적인 구성/구조 보다는 각 개별단어 간의 '의존관계' 또는 '수식관계' 와 같은 단어간 관계를 파악하는 것이 목적인 NLP Task
* 문장 해석의 모호성을 없애기 위해 Parsing 을 한다.




🔹 **3-(1)** Dependency Parsing by spacy library


* [basic](https://frhyme.github.io/python-lib/nlp_spacy_1/#navigating-parse-tree) 👉 dependecy parsing 부분만 필수
* .dep_ 메서드



In [27]:
doc = nlp("Autonomous cars shift insurance liability toward manufacturers")

noun_chunks = doc.noun_chunks
print(type(noun_chunks))
noun_chunk = list(noun_chunks)[0]
print(type(noun_chunk))
token = noun_chunk[0]
print(type(token))

print("=="*30)
print("""
Text: The original noun chunk text.
Root text: The original text of the word connecting the noun chunk to the rest of the parse.
Root dep: Dependency relation connecting the root to its head.
Root head text: The text of the root token's head.
""".strip())
print("=="*30)
str_format = "{:>25}"*4
for chunk in doc.noun_chunks:
    print(str_format.format(chunk.text, chunk.root.text, chunk.root.dep_, chunk.root.head.text))

<class 'generator'>
<class 'spacy.tokens.span.Span'>
<class 'spacy.tokens.token.Token'>
Text: The original noun chunk text.
Root text: The original text of the word connecting the noun chunk to the rest of the parse.
Root dep: Dependency relation connecting the root to its head.
Root head text: The text of the root token's head.
          Autonomous cars                     cars                    nsubj                    shift
      insurance liability                liability                     dobj                    shift
            manufacturers            manufacturers                     pobj                   toward


In [28]:
## navigiting parse tree
doc = nlp("Autonomous cars shift insurance liability toward manufacturers")
for tok in doc:
    print(tok.text)
    children = list(tok.children)
    print('children:', children, 'head:', tok.head if tok.head != tok else "!this is root node")
    print("=="*16)

Autonomous
children: [] head: cars
cars
children: [Autonomous] head: shift
shift
children: [cars, liability] head: !this is root node
insurance
children: [] head: liability
liability
children: [insurance, toward] head: shift
toward
children: [manufacturers] head: liability
manufacturers
children: [] head: toward


In [30]:
import networkx as nx
import matplotlib.pyplot as plt 

nG = nx.Graph()
doc[2] ## root node

def add_n_to_g(inputG, tok):
    inputG.add_node(tok)
    children = list(tok.children)
    if children != []:
        inputG.add_nodes_from(children)
        for c in children:
            inputG.add_edges_from([(tok, c, {'dependency':c.dep_})])
            add_n_to_g(inputG, c)
            
add_n_to_g(nG, doc[2])
print(nG.nodes(data=True))
print("=="*20)
for e in nG.edges(data=True):
    print(f"{e[0]}, {e[1]}, ### dependency: {e[2]['dependency']}")

[(shift, {}), (cars, {}), (liability, {}), (Autonomous, {}), (insurance, {}), (toward, {}), (manufacturers, {})]
shift, cars, ### dependency: nsubj
shift, liability, ### dependency: dobj
cars, Autonomous, ### dependency: amod
liability, insurance, ### dependency: compound
liability, toward, ### dependency: prep
toward, manufacturers, ### dependency: pobj


🔹 **3-(2)** Spacy (kaggle) 

* 캐글 노트북 환경에서 실습해보는 것을 권장드립니다!

* [kaggle_spaCy](https://www.kaggle.com/code/nirant/hitchhiker-s-guide-to-nlp-in-spacy) 👉 필수
  * 도날드 트럼프 트위터 트윗 내용 데이터 분석


👀 **노트북 키포인트** 
  1. spacy.display 메서드를 사용한 NER 시각화 
  2. Tagging 을 통한 트럼프 트윗 분석 : noun_chunks 는 dependency graph를 고려하여, noun phrase를 뽑아준다. 
  3. [spacy Match](https://yujuwon.tistory.com/entry/spaCy-%EC%82%AC%EC%9A%A9%ED%95%98%EA%B8%B0-Rule-based-Matching) : 직접 문장/단어 패턴을 등록하여 parsing
  4. Question and answering task using Dependency Parsing
    * spacy display :  ``style = 'dep'``
    * .dep_
